# Method 1: Deploying a Cognite Function via the SDK



## 1. Authentication and Client Creation

Here we'll create the cognite client using Client credentials. We need to supply the tenant_id, client_id and client secret along with cluster name and project name to create the cognite client. Once this client is created, this will be the main way to communicate with the Cognite Data Fusion.

Import and create a client to communicate with the cognite data platform (CDF)

In [2]:
#from cognite.client import CogniteClient
from msal import PublicClientApplication
from cognite.experimental import CogniteClient


YOUR_NAME = "Jack"

# Contact Project Administrator to get these
TENANT_ID="48d5043c-cf70-4c49-881c-c638f5796997"
CLIENT_ID="fab52bb5-9de2-4f9e-aefa-712da4b5fe00"
CDF_CLUSTER="westeurope-1"
COGNITE_PROJECT="ds-basics"

SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000

app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)
creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)

TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-client-interactive"
    )

Opening in existing browser session.


libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)


## 2. Creating the handle function

As shown in the Examples https://github.com/cognitedata/cognite-function-examples/blob/main/examples/01-creating-a-function-from-a-notebook/create-cognite-function.ipynb , we need to create a function called "handle" and put the required calculation code/steps inside it.

The `thermal_resistance` function that we defined in the previous notebook has been moved to the `tools.py` file for simplicity.

In [2]:
def handle(client, data=None, secrets=None, function_call_info=None):
    
    from tools import thermal_resistance
    import datetime
    from datetime import timedelta
    """Handler Function to be Run/Deployed
    Args:
        client : Cognite Client (not needed, it's availble to it, when deployed)
        data : data needed by function
        secrets : Any secrets it needs
        function_call_info : any other information about function

    Returns:
        response : response or result from the function 
    """
    
    ts_exids = ['pi:163657','pi:163658','pi:160887','pi:191092','pi:163374','pi:160184']
    column_names = ["T_cold_IN","T_cold_OUT","T_hot_IN","T_hot_OUT","Flow_cold","Flow_hot"]
    # Retrieve the data
    start_date = datetime.datetime(2018, 8, 1)
    end_date = start_date + timedelta(days=10)
    df = client.datapoints.retrieve_dataframe(external_id=ts_exids,
                                                        aggregates=['average'],
                                                        granularity='6h',
                                                        start=start_date,
                                                        end=end_date,
                                                        include_aggregate_name=False
                                                        )
    df.fillna(method="ffill", inplace=True)
    df.columns = column_names
    # Calculate the Thermal resistance
    df['TR'] = df.apply(lambda x:thermal_resistance(x),axis=1)
    # Return the result as json
    result = df[['TR']].to_json()
    return result

Before we deploy this Python-function as a Cognite Function, we can verify that it works locally by passing in our previously instantiated CogniteClient and a data-dictionary.

In [3]:
handle(client)

'{"TR":{"1533081600000":0.0000003966,"1533103200000":0.0000003973,"1533124800000":0.0000003986,"1533146400000":0.000000404,"1533168000000":0.0000004002,"1533189600000":0.0000003968,"1533211200000":0.0000003998,"1533232800000":0.0000003995,"1533254400000":0.0000003981,"1533276000000":0.0000004014,"1533297600000":0.0000004022,"1533319200000":0.0000003718,"1533340800000":0.0000003796,"1533362400000":0.0000003757,"1533384000000":0.0000003942,"1533405600000":0.0000004016,"1533427200000":0.0000003976,"1533448800000":0.000000398,"1533470400000":0.0000004012,"1533492000000":0.0000003983,"1533513600000":0.0000003908,"1533535200000":0.0000003976,"1533556800000":0.0000003976,"1533578400000":0.0000004013,"1533600000000":0.0000003962,"1533621600000":0.0000003976,"1533643200000":0.0000003979,"1533664800000":0.0000003988,"1533686400000":0.0000004085,"1533708000000":0.0000004102,"1533729600000":0.0000004098,"1533751200000":0.0000003997,"1533772800000":0.0000004035,"1533794400000":0.0000004058,"1533816

## 3. Deploying the Function to CDF

If the calculation is simple and basic, doesn't need any external dependencies, it can be created & deployed directly using python-sdk, by passing handle function as a parameter ( for function_handle) in functions.create command, as shown below.

In [4]:
function_external_id = f"hx_thermal_perf_{YOUR_NAME}"
function_name = "Thermal Performance Calculation"

In [5]:
# Delete If the same function exists already
if client.functions.retrieve(external_id=function_external_id):
    client.functions.delete(external_id=function_external_id)
    print('Function found and deleted')
    
print('You are good to go!')


You are good to go!


In [6]:
function = client.functions.create(
    name=function_name,
    external_id=function_external_id,
    function_handle=handle,
    description="Function to Calculate Thermal Performance of Discharge Coolers",
    runtime="py39")

This might take few minutes to deploy before function is ready to run. You can check the status as follows:

In [7]:
import time

def status_check(function):

    start_time = time.time()
    # Repeat until status is ready
    while function.status != "Ready":

        function.update()

        time_elapsed = int(time.time()-start_time)

        print(function.status + f'. Waiting for {time_elapsed} seconds', end='\r')

        if function.status == "Failed":
            print("Failed to deploy function")
            break

        time.sleep(5)
    else:
        print(f"Function is successfully deployed. Wait time: {time_elapsed} seconds.")
        
status_check(function)

Function is successfully deployed. Wait time: 233 seconds.


If the function has deployed succesfully, then we can try calling it with the SDK.

In [10]:
call = function.call()
call

,value
id,3924353550912508
start_time,1659542833146
end_time,1659542837685
status,Failed
function_id,1869427694501493


Looks like the function has failed, let's investigate the logs to see what has happened.

In [11]:
call.get_logs()

,timestamp,message
0,1659542837177,Traceback (most recent call last):
1,1659542837177,"File ""/home/site/wwwroot/function/__init__.p..."
2,1659542837177,result = handle(*function_argument_values)
3,1659542837177,"File ""/home/site/wwwroot/function/handler.py..."
4,1659542837177,from tools import thermal_resistance
5,1659542837177,ModuleNotFoundError: No module named 'tools'
6,1659542837177,MemoryError: The function ran out of memory.


As logs say "ModuleNotFoundError: No module named 'tools'", it looks like it can't find the thermal resistance function. We can fix this in the next section by making sure that our function has all the packages that it needs to run properly.


## 4. Uploading a folder to Cognite Functions

We have the neat option of uploading the contents of an entire folder to our Cognite Function. To this end, let's create a new folder `hx-thermal-resistance` and put all our code in there.

In this folder you should have the following:
* handler.py
* tools.py
* requirements.txt

The 4 notebook cells below will automatically create the folder and the code files for you.

* Create directory
* Copy tools.py over to the folder
* Create handler.py in the folder
* Create requirements.txt in the folder

In [12]:
%%sh

mkdir ../hx-thermal-resistance


In [13]:
%%sh

cp tools.py ../hx-thermal-resistance/

In [14]:
%%sh

echo """
import datetime
from datetime import timedelta
from tools import thermal_resistance
def handle(client, data=None, secrets=None, function_call_info=None):
    '''Handler Function to be Run/Deployed
    Args:
        client : Cognite Client (not needed, it's availble to it, when deployed)
        data : data needed by function
        secrets : Any secrets it needs
        function_call_info : any other information about function

    Returns:
        response : response or result from the function 
    '''
    
    ts_exids = ['pi:163657','pi:163658','pi:160887','pi:191092','pi:163374','pi:160184']
    column_names = ['T_cold_IN','T_cold_OUT','T_hot_IN','T_hot_OUT','Flow_cold','Flow_hot']
    # Retrieve the data
    start_date = datetime.datetime(2018, 8, 1)
    end_date = start_date + timedelta(days=10)
    df = client.datapoints.retrieve_dataframe(external_id=ts_exids,
                                                        aggregates=['average'],
                                                        granularity='6h',
                                                        start=start_date,
                                                        end=end_date,
                                                        include_aggregate_name=False
                                                        )
    df.fillna(method='ffill', inplace=True)
    df.columns = column_names
    # Calculate the Thermal resistance
    df['TR'] = df.apply(lambda x:thermal_resistance(x),axis=1)
    # Return the result as json
    result = df[['TR']].to_json()
    return result
""" > ../hx-thermal-resistance/handler.py

In [15]:
%%sh

echo """pandas >=1.4.3""" > ../hx-thermal-resistance/requirements.txt

In [16]:
# Delete the function we deployed to avoid duplication errors
client.functions.delete(external_id=function_external_id)

In [17]:
function = client.functions.create(
    name=function_name,
    external_id=function_external_id,
    folder="../hx-thermal-resistance/",  # We will upload this directory
    description="Function to Calculate Thermal Performance of Discharge Coolers",
    owner="Jack Reacher",
)

In [18]:
# Check the status if it has been deployed
status_check(function)

Function is successfully deployed. Wait time: 279 seconds.


In [19]:
# Check if the function call has been successful
call = function.call()
call

,value
id,2659944705674703
start_time,1659543151585
end_time,1659543163396
status,Completed
function_id,6363084569903198


In [20]:
# Get response
call.get_response()

'{"TR":{"1533081600000":0.0000003966,"1533103200000":0.0000003973,"1533124800000":0.0000003986,"1533146400000":0.000000404,"1533168000000":0.0000004002,"1533189600000":0.0000003968,"1533211200000":0.0000003998,"1533232800000":0.0000003995,"1533254400000":0.0000003981,"1533276000000":0.0000004014,"1533297600000":0.0000004022,"1533319200000":0.0000003718,"1533340800000":0.0000003796,"1533362400000":0.0000003757,"1533384000000":0.0000003942,"1533405600000":0.0000004016,"1533427200000":0.0000003976,"1533448800000":0.000000398,"1533470400000":0.0000004012,"1533492000000":0.0000003983,"1533513600000":0.0000003908,"1533535200000":0.0000003976,"1533556800000":0.0000003976,"1533578400000":0.0000004013,"1533600000000":0.0000003962,"1533621600000":0.0000003976,"1533643200000":0.0000003979,"1533664800000":0.0000003988,"1533686400000":0.0000004085,"1533708000000":0.0000004102,"1533729600000":0.0000004098,"1533751200000":0.0000003997,"1533772800000":0.0000004035,"1533794400000":0.0000004058,"1533816

## 5. Running a Cognite Function on a schedule

When creating function in CDF UI, we can also create the schedule,
- for hourly schedule, enter this cron expression `0 * * * *`
- for daily `0 0 * * *` 
- for every minute `* * * * *` 

In [7]:
function = client.functions.retrieve(1253610691220398)

In [8]:
schedule = client.functions.schedules.create(
    name="run-function-every-day",
    cron_expression="0 0 * * *",  # the cron expression runs every day
    function_id=function.id,  # we specify the ID of the function we want to schedule
    data={},  # this is the data we wish to call the function with (e.g. time range for calculation)
    description="This schedule runs the function every minute"
    
)

CogniteAPIError: A session nonce must be provided when creating a function schedule. | code: 400 | X-Request-ID: 9a316d5b-04c6-93c0-a26f-4a70e53ead61

In [6]:
client.functions.list()

,id,name,owner,status,file_id,function_path,created_time,cpu,memory,runtime,metadata,external_id,description
0,1253610691220398,Test Function,Andris Piebalgs,Ready,2869365677300270,handler.py,1659543213430,0.25,1.0,py38,{},NaN,NaN
1,6269662495170826,Thermal Performance Function with UI,NaN,Ready,410249806612463,handler.py,1658837552859,0.25,1.0,py38,{},NaN,NaN
2,6329621345240516,Thermal Performance Forecasting Function,Choukha Ram,Ready,4260666617221294,handler.py,1658842545691,1.00,1.5,py38,{},thermal-perf-forecast,Function to Calculate & Forecast Thermal Perfo...
3,6363084569903198,Thermal Performance Calculation,Jack Reacher,Ready,1245301260806451,handler.py,1659542856482,1.00,1.5,py38,{},hx_thermal_perf_Jack,Function to Calculate Thermal Performance of D...
4,6999183115437061,Thermal Performance Calculation and Save Function,Choukha Ram,Ready,2705633949196009,handler.py,1658839942369,1.00,1.5,py38,{},thermal-perf-save-calc,Function to Calculate & Save Thermal Performan...
5,7693341838777347,Thermal Performance Calculation and Save Function,Choukha Ram,Ready,3473668269847550,handler.py,1659545818538,1.00,1.5,py38,{},thermal-perf-save-calc-Chouka-Ram,Function to Calculate & Save Thermal Performan...


Deleting 